<a href="https://colab.research.google.com/github/TummalaSharmila/HT/blob/Assignment-4/A4_HT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import numpy as np
import bz2

# Function to load sequences from a compressed .bz2 file
def load_sequences(filename):
    sequences = {}
    try:
        with bz2.open(filename, 'rt') as f:
            for line in f:
                parts = line.strip().split(' \\ ')
                if len(parts) == 2:
                    gene_id, sequence = parts[0].strip(), parts[1].strip()
                    sequences[gene_id] = sequence
                else:
                    print(f"Skipping malformed line: {line.strip()}")
    except Exception as e:
        print(f"Error reading or parsing file: {e}")
    print(f"Total sequences loaded: {len(sequences)}")
    return sequences

# Function to parse the custom counts matrix from the assignment file
def parse_counts_matrix(filepath):
    base_order = ['a', 'c', 'g', 't']
    counts_dict = {b: [] for b in base_order}
    with open(filepath) as f:
        for line in f:
            line = line.strip().lower()
            if not line or line[0] not in base_order:
                continue
            parts = line.replace('|', '').split()
            base = parts[0]
            counts = list(map(int, parts[1:]))
            counts_dict[base].extend(counts)
    return np.array([counts_dict[b] for b in base_order])

# Compute PWM from counts with pseudocount
def compute_pwm(counts):
    pseudocount = 1
    counts_pseudo = counts + pseudocount
    total = counts_pseudo.sum(axis=0)
    frequency_matrix = counts_pseudo / total
    background_freq = 0.25
    return np.log2(frequency_matrix / background_freq)

# Compute adjusted frequency matrix F' with pseudocounts
def compute_adjusted_frequency(counts):
    pseudocount = 1
    return (counts + pseudocount) / (counts.sum(axis=0) + 4 * pseudocount)

# Scan sequences using PWM and return top N hits
def scan_sequences(pwm, sequences, top_n=30):
    base_mapping = {'a': 0, 'c': 1, 'g': 2, 't': 3}
    motif_len = pwm.shape[1]
    scores = []
    for gene_id, seq in sequences.items():
        seq = seq.lower()
        if len(seq) < motif_len:
            print(f"Skipping {gene_id} due to short sequence.")
            continue
        for i in range(len(seq) - motif_len + 1):
            subseq = seq[i:i+motif_len]
            try:
                score = sum(pwm[base_mapping[b], j] for j, b in enumerate(subseq) if b in base_mapping)
                scores.append((gene_id, score, subseq, i))
            except KeyError:
                continue
    scores.sort(key=lambda x: x[1], reverse=True)
    return scores[:top_n]

# File paths
bzipped_path = 'E_coli_K12_MG1655.400_50.bz2'
count_matrix_file = 'argR-counts-matrix.txt'

# Main workflow
sequences = load_sequences(bzipped_path)
if not sequences:
    print("No sequences were loaded. Please check the input file.")
else:
    counts = parse_counts_matrix(count_matrix_file)
    pwm = compute_pwm(counts)
    adjusted_freq = compute_adjusted_frequency(counts)

    print("\nPWM (Position Weight Matrix):")
    print(pwm)
    print("\nAdjusted Frequency Matrix F'(b, j):")
    print(adjusted_freq)

    top_sites = scan_sequences(pwm, sequences)
    print(f"\nTop {len(top_sites)} binding sites:")
    for gene_id, score, subseq, pos in top_sites:
        print(f"{gene_id} | Score: {score:.2f} | Pos: {pos} | Site: {subseq}")


Total sequences loaded: 4319

PWM (Position Weight Matrix):
[[ 0.21572869  0.74624341  1.50523531  0.36773178 -0.63226822 -1.36923381
   1.50523531  1.50523531 -0.95419631  0.50523531  0.21572869 -0.36923381
   0.04580369  1.74624341 -0.63226822 -1.36923381 -1.36923381  1.74624341]
 [ 0.04580369 -0.63226822 -1.95419631 -0.14684139 -1.36923381 -0.95419631
  -0.95419631 -0.95419631 -1.95419631 -2.95419631 -1.36923381 -2.95419631
   0.04580369 -2.95419631 -0.95419631 -0.95419631  1.68965988 -2.95419631]
 [-0.95419631 -1.36923381 -1.95419631  0.21572869 -1.36923381  1.50523531
  -1.36923381 -1.36923381 -2.95419631 -1.95419631 -2.95419631 -1.95419631
  -2.95419631 -1.95419631 -2.95419631  1.04580369 -2.95419631 -1.36923381]
 [ 0.36773178  0.36773178 -0.63226822 -0.63226822  1.36773178 -1.95419631
  -1.95419631 -1.95419631  1.63076619  1.13326653  1.21572869  1.50523531
   0.85315861 -1.95419631  1.43812111  0.04580369 -1.95419631 -2.95419631]]

Adjusted Frequency Matrix F'(b, j):
[[0.290322